In [ ]:
# cityscapes_color_to_yoloseg_with_images.py
# X_gtFine_color.png (maske) + X_leftImg8bit.png (görüntü) çiftlerinden
# YOLO-Seg etiketleri üretir ve görüntüyü labels'ın yanındaki images klasörüne kopyalar/taşır.

import cv2, json, shutil, os, numpy as np
from pathlib import Path

# =================== AYARLAR ===================
ROOT      = Path(r"C:\Users\tahay\OneDrive\Desktop\Cityscapes - Kopya\leftImg8bit_trainvaltest\leftImg8bit\train\aachen")
OUT_DIR   = ROOT / "yolo_out"          # çıktı kökü
WRITE_BBOX_TOO = False                 # True -> aynı konturlardan bbox .txt de üret
MIN_AREA = 50                          # çok küçük parçaları at
APPROX_EPS = 0.003                     # kontur sadeleştirme (Douglas–Peucker)
ROUND_TO = 0                           # antialias varsa 8 gibi bir değer verin; 0=kapalı
USE_CITYSCAPES_PALETTE = True          # True: Cityscapes 19-sınıf paleti; False: maskeden otomatik keşif

# Görüntüyü kopyala mı/taşı mı?
COPY_IMAGES = True     # True: kopyala
MOVE_IMAGES = False    # True: TAŞI (kopyalama yerine); dikkat: orijinali yerinden alır

# === Cityscapes trainId (0..18) isimleri ===
CS_TRAIN_NAMES = [
    "road","sidewalk","building","wall","fence","pole","traffic light","traffic sign",
    "vegetation","terrain","sky","person","rider","car","truck","bus","train","motorcycle","bicycle"
]

# === Cityscapes color (RGB) -> trainId (19 sınıf) ===
CS_COLOR_TO_TRAINID = {
    (128, 64,128): 0,   # road
    (244, 35,232): 1,   # sidewalk
    ( 70, 70, 70): 2,   # building
    (102,102,156): 3,   # wall
    (190,153,153): 4,   # fence
    (153,153,153): 5,   # pole
    (250,170, 30): 6,   # traffic light
    (220,220,  0): 7,   # traffic sign
    (107,142, 35): 8,   # vegetation
    (152,251,152): 9,   # terrain
    ( 70,130,180): 10,  # sky
    (220, 20, 60): 11,  # person
    (255,  0,  0): 12,  # rider
    (  0,  0,142): 13,  # car
    (  0,  0, 70): 14,  # truck
    (  0, 60,100): 15,  # bus
    (  0, 80,100): 16,  # train
    (  0,  0,230): 17,  # motorcycle
    (119, 11, 32): 18,  # bicycle
}

IGNORE_COLORS = {(0,0,0)}  # 'void' vb.

# =================== YARDIMCI FONKSİYONLAR ===================
def to_hex(c): return "#{:02X}{:02X}{:02X}".format(*c)

def simplify(cnt, eps_ratio=APPROX_EPS):
    peri = cv2.arcLength(cnt, True)
    eps = eps_ratio * peri
    return cv2.approxPolyDP(cnt, eps, True)

def poly_to_yolo(cnt, w, h):
    pts = cnt.reshape(-1,2).astype(np.float32)
    pts[:,0] /= w; pts[:,1] /= h
    return pts.flatten().tolist()

def bbox_to_yolo(cnt, w, h):
    x,y,wc,hc = cv2.boundingRect(cnt)
    return [(x+wc/2)/w, (y+hc/2)/h, wc/w, hc/h]

def unique_colors_fast(mask):
    if ROUND_TO > 0:
        mask = (mask//ROUND_TO)*ROUND_TO
    flat = mask.reshape(-1,3)
    uniq = np.unique(flat, axis=0)
    return [tuple(int(v) for v in row) for row in uniq]

def robust_move_or_copy(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if MOVE_IMAGES:
        try:
            src.replace(dst)  # hızlı taşımaya çalış
        except Exception:
            # farklı disk/birim ise: kopyala + sil
            shutil.copy2(src, dst)
            try:
                src.unlink()
            except Exception:
                pass
    elif COPY_IMAGES:
        if not dst.exists():
            shutil.copy2(src, dst)

def process_pair(mask_path: Path, img_path: Path, labels_dir: Path, images_dir: Path,
                 color2cls: dict, auto_palette: bool):
    img  = cv2.imread(str(img_path),  cv2.IMREAD_COLOR)
    mask = cv2.imread(str(mask_path), cv2.IMREAD_COLOR)
    if img is None or mask is None:
        print(f"[WARN] Okunamadı: {img_path.name} / {mask_path.name}")
        return 0

    H, W = img.shape[:2]
    if mask.shape[:2] != (H, W):
        mask = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)
    if ROUND_TO > 0:
        mask = (mask//ROUND_TO)*ROUND_TO

    # Otomatik palet: maske içinde görülen renklere id ata
    if auto_palette:
        for c in unique_colors_fast(mask):
            if c in IGNORE_COLORS: 
                continue
            if c not in color2cls:
                color2cls[c] = len(color2cls)

    base = img_path.stem  # ..._leftImg8bit
    out_txt = labels_dir / f"{base}.txt"
    seg_lines, bbox_lines = [], []

    for rgb, cid in color2cls.items():
        if rgb in IGNORE_COLORS: 
            continue
        m = cv2.inRange(mask, np.array(rgb), np.array(rgb))
        contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        for cnt in contours:
            if cv2.contourArea(cnt) < MIN_AREA:
                continue
            cnt = simplify(cnt)
            if len(cnt) < 3:
                continue
            seg = poly_to_yolo(cnt, W, H)
            if len(seg) < 6:
                continue
            seg_lines.append(str(cid) + " " + " ".join(f"{v:.6f}" for v in seg))
            if WRITE_BBOX_TOO:
                cx,cy,bw,bh = bbox_to_yolo(cnt, W, H)
                bbox_lines.append(f"{cid} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    out_txt.write_text("\n".join(seg_lines))
    if WRITE_BBOX_TOO and bbox_lines:
        (labels_dir / f"{base}_bbox.txt").write_text("\n".join(bbox_lines))

    # === İLGİLİ GÖRÜNTÜYÜ images/ altına kopyala/taşı ===
    out_img = images_dir / img_path.name  # aynı dosya adıyla
    robust_move_or_copy(img_path, out_img)

    return len(seg_lines)

# =================== ANA AKIŞ ===================
def main():
    labels_dir = OUT_DIR / "labels"
    images_dir = OUT_DIR / "images"
    labels_dir.mkdir(parents=True, exist_ok=True)
    images_dir.mkdir(parents=True, exist_ok=True)

    # Bu klasördeki tüm gtFine_color maskeleri tara
    mask_paths = sorted(ROOT.glob("*_gtFine_color.png"))
    total_objs = processed = 0

    # Palet seçimi
    if USE_CITYSCAPES_PALETTE:
        color2cls = dict(CS_COLOR_TO_TRAINID)
        class_names = CS_TRAIN_NAMES
        auto_palette = False
    else:
        color2cls    = {}
        class_names  = []
        auto_palette = True

    for m in mask_paths:
        base = m.name.replace("_gtFine_color.png", "")
        img  = ROOT / f"{base}_leftImg8bit.png"
        if not img.exists():
            print(f"[WARN] Görüntü yok: {img.name}")
            continue
        n = process_pair(m, img, labels_dir, images_dir, color2cls, auto_palette)
        total_objs += n
        processed  += 1

    # dataset.yaml ve renk eşlemeleri
    names = (class_names if USE_CITYSCAPES_PALETTE
             else [f"class_{i}_{to_hex(c)}" for c,i in sorted(color2cls.items(), key=lambda x:x[1])])

    (OUT_DIR/"dataset.yaml").write_text("names:\n" + "".join([f"  - {n}\n" for n in names]))

    with open(OUT_DIR/"color_map.json", "w") as f:
        json.dump({to_hex(k): v for k,v in sorted(color2cls.items(), key=lambda x:x[1])}, f, indent=2)

    print(f"Tamamlandı. Çift sayısı: {processed}, yazılan poligon: {total_objs}")
    print(f"Görüntüler: {images_dir}")
    print(f"Etiketler : {labels_dir}")

if __name__ == "__main__":
    main()
